<a href="https://colab.research.google.com/github/namjad381/Machine-Learning-AMD-project/blob/main/Glasses_or_no_Glasses_in_Spark_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download the Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload() #Upload kaggle username and key



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"namjad","key":"5f515f39dff4a6f911f93015f24f0b69"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              237KB  2021-11-02 21:56:08          15266  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           9289  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           4143  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-10-22 10:48:21           3400  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [ ]:
! kaggle datasets download -d 'jeffheaton/glasses-or-no-glasses' -f train.csv

  0% 0.00/7.41M [00:00<?, ?B/s]
100% 7.41M/7.41M [00:00<00:00, 68.5MB/s]


In [ ]:
! kaggle datasets download -d 'jeffheaton/glasses-or-no-glasses' -f test.csv

  0% 0.00/844k [00:00<?, ?B/s]
100% 844k/844k [00:00<00:00, 51.5MB/s]


In [ ]:
! mkdir glasses

In [ ]:
!unzip train.csv.zip -d glasses

Archive:  train.csv.zip
  inflating: glasses/train.csv       


In [ ]:
!unzip test.csv.zip -d glasses

Archive:  test.csv.zip
  inflating: glasses/test.csv        


# Import Libraries


In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 53.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=4e88a2f402f64892cff7638e37f5f840fb07911be5fe130860b2e7657297ed3a
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from pyspark.ml.feature import StandardScaler
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import plotly
plotly.__version__

'4.4.1'

# Dataset Import

In [ ]:
spark=SparkSession.builder.appName('practise').getOrCreate()
spark

In [ ]:
df_pyspark=spark.read.csv('/content/glasses/train.csv', header=True, inferSchema=True)

In [ ]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [ ]:
df_pyspark.printSchema()

root
 |-- id: integer (nullable = true)
 |-- v1: double (nullable = true)
 |-- v2: double (nullable = true)
 |-- v3: double (nullable = true)
 |-- v4: double (nullable = true)
 |-- v5: double (nullable = true)
 |-- v6: double (nullable = true)
 |-- v7: double (nullable = true)
 |-- v8: double (nullable = true)
 |-- v9: double (nullable = true)
 |-- v10: double (nullable = true)
 |-- v11: double (nullable = true)
 |-- v12: double (nullable = true)
 |-- v13: double (nullable = true)
 |-- v14: double (nullable = true)
 |-- v15: double (nullable = true)
 |-- v16: double (nullable = true)
 |-- v17: double (nullable = true)
 |-- v18: double (nullable = true)
 |-- v19: double (nullable = true)
 |-- v20: double (nullable = true)
 |-- v21: double (nullable = true)
 |-- v22: double (nullable = true)
 |-- v23: double (nullable = true)
 |-- v24: double (nullable = true)
 |-- v25: double (nullable = true)
 |-- v26: double (nullable = true)
 |-- v27: double (nullable = true)
 |-- v28: double (nullab

In [ ]:
df_pyspark.show()

+---+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-----

#Feature preprocessing




In [ ]:
from pyspark.sql.functions import col
df_pyspark = df_pyspark.select(*(col(c).cast('float').alias(c) for c in df_pyspark.columns))

In [ ]:
df_pyspark.printSchema()

root
 |-- id: float (nullable = true)
 |-- v1: float (nullable = true)
 |-- v2: float (nullable = true)
 |-- v3: float (nullable = true)
 |-- v4: float (nullable = true)
 |-- v5: float (nullable = true)
 |-- v6: float (nullable = true)
 |-- v7: float (nullable = true)
 |-- v8: float (nullable = true)
 |-- v9: float (nullable = true)
 |-- v10: float (nullable = true)
 |-- v11: float (nullable = true)
 |-- v12: float (nullable = true)
 |-- v13: float (nullable = true)
 |-- v14: float (nullable = true)
 |-- v15: float (nullable = true)
 |-- v16: float (nullable = true)
 |-- v17: float (nullable = true)
 |-- v18: float (nullable = true)
 |-- v19: float (nullable = true)
 |-- v20: float (nullable = true)
 |-- v21: float (nullable = true)
 |-- v22: float (nullable = true)
 |-- v23: float (nullable = true)
 |-- v24: float (nullable = true)
 |-- v25: float (nullable = true)
 |-- v26: float (nullable = true)
 |-- v27: float (nullable = true)
 |-- v28: float (nullable = true)
 |-- v29: float (nu

In [ ]:
from pyspark.sql.functions import isnan, when, count, col
df_pyspark.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_pyspark.columns]).show()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----



*     Combine all feature columns into a single feature vector





In [ ]:
cols=df_pyspark.columns
cols.remove('glasses') 
cols.remove('id')
assembler = VectorAssembler(inputCols=cols,outputCol='features')
data = assembler.transform(df_pyspark)
data.select('features', 'glasses').show() #we can apply truncate=False to see all features

+--------------------+-------+
|            features|glasses|
+--------------------+-------+
|[0.37797001004219...|    0.0|
|[0.07609000056982...|    1.0|
|[1.19391000270843...|    1.0|
|[1.34949004650115...|    0.0|
|[-0.0351199992001...|    0.0|
|[-0.4617699980735...|    1.0|
|[-0.4787899851799...|    1.0|
|[-0.2777999937534...|    1.0|
|[0.15765999257564...|    0.0|
|[0.57302999496459...|    1.0|
|[-0.4857900142669...|    0.0|
|[-0.0151599999517...|    1.0|
|[0.39533999562263...|    0.0|
|[1.67018997669219...|    1.0|
|[0.56269997358322...|    1.0|
|[0.41036000847816...|    0.0|
|[0.25106000900268...|    1.0|
|[-0.1004000008106...|    0.0|
|[1.44832003116607...|    1.0|
|[1.03922998905181...|    0.0|
+--------------------+-------+
only showing top 20 rows



In [ ]:
standardscaler = StandardScaler(inputCol="features", outputCol="features_scaled")
data=standardscaler.fit(data).transform(data)

In [ ]:
data.select("features", "glasses", "features_scaled").show()

+--------------------+-------+--------------------+
|            features|glasses|     features_scaled|
+--------------------+-------+--------------------+
|[0.37797001004219...|    0.0|[0.49099010762590...|
|[0.07609000056982...|    1.0|[0.09884233292705...|
|[1.19391000270843...|    1.0|[1.55091140871210...|
|[1.34949004650115...|    0.0|[1.75301279352225...|
|[-0.0351199992001...|    0.0|[-0.0456215353836...|
|[-0.4617699980735...|    1.0|[-0.5998478583717...|
|[-0.4787899851799...|    1.0|[-0.6219571397409...|
|[-0.2777999937534...|    1.0|[-0.3608673842039...|
|[0.15765999257564...|    0.0|[0.20480327715517...|
|[0.57302999496459...|    1.0|[0.74437667387720...|
|[-0.4857900142669...|    0.0|[-0.6310503083615...|
|[-0.0151599999517...|    1.0|[-0.0196931232905...|
|[0.39533999562263...|    0.0|[0.51355404355471...|
|[1.67018997669219...|    1.0|[2.16960799699498...|
|[0.56269997358322...|    1.0|[0.73095778302591...|
|[0.41036000847816...|    0.0|[0.53306532099086...|
|[0.25106000

In [ ]:
assembled_data = data.select("features_scaled", "glasses")
assembled_data.show()


+--------------------+-------+
|     features_scaled|glasses|
+--------------------+-------+
|[0.49099010762590...|    0.0|
|[0.09884233292705...|    1.0|
|[1.55091140871210...|    1.0|
|[1.75301279352225...|    0.0|
|[-0.0456215353836...|    0.0|
|[-0.5998478583717...|    1.0|
|[-0.6219571397409...|    1.0|
|[-0.3608673842039...|    1.0|
|[0.20480327715517...|    0.0|
|[0.74437667387720...|    1.0|
|[-0.6310503083615...|    0.0|
|[-0.0196931232905...|    1.0|
|[0.51355404355471...|    0.0|
|[2.16960799699498...|    1.0|
|[0.73095778302591...|    1.0|
|[0.53306532099086...|    0.0|
|[0.32613164421968...|    1.0|
|[-0.1304214776144...|    0.0|
|[1.88139479081848...|    1.0|
|[1.34997917987108...|    0.0|
+--------------------+-------+
only showing top 20 rows



In [ ]:
assembled_data = assembled_data.withColumnRenamed("features_scaled","features")
assembled_data = assembled_data.withColumnRenamed("glasses","label")
assembled_data.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.49099010762590...|  0.0|
|[0.09884233292705...|  1.0|
|[1.55091140871210...|  1.0|
|[1.75301279352225...|  0.0|
|[-0.0456215353836...|  0.0|
|[-0.5998478583717...|  1.0|
|[-0.6219571397409...|  1.0|
|[-0.3608673842039...|  1.0|
|[0.20480327715517...|  0.0|
|[0.74437667387720...|  1.0|
|[-0.6310503083615...|  0.0|
|[-0.0196931232905...|  1.0|
|[0.51355404355471...|  0.0|
|[2.16960799699498...|  1.0|
|[0.73095778302591...|  1.0|
|[0.53306532099086...|  0.0|
|[0.32613164421968...|  1.0|
|[-0.1304214776144...|  0.0|
|[1.88139479081848...|  1.0|
|[1.34997917987108...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [ ]:
# Split the data into train and test
splits =assembled_data.randomSplit([0.7, 0.3], 1234)
train = splits[0]
test = splits[1]

In [ ]:
train.count()

3215

#Model I


*   HyperParameter Tuning (ParamGrid)
*   Cross Validation
*   Prediction






In [56]:
## Parameter Grid
layers = [512,256,128,2]
mlp = MultilayerPerceptronClassifier(layers = layers, seed = 1)
paramGrid_mlp = ParamGridBuilder() \
    .addGrid(mlp.blockSize, {64,128,256}) \
    .addGrid(mlp.maxIter, {50,100,150}) \
    .build()

In [57]:
## CrossValidation using 5-Folds and Binary Classifier
mlp_cv = CrossValidator(estimator=mlp,
                          estimatorParamMaps=paramGrid_mlp,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5)  

In [58]:
paramGrid_mlp

[{Param(parent='MultilayerPerceptronClassifier_c88cde667a78', name='blockSize', doc='block size for stacking input data in matrices. Data is stacked within partitions. If block size is more than remaining data in a partition then it is adjusted to the size of this data.'): 64,
  Param(parent='MultilayerPerceptronClassifier_c88cde667a78', name='maxIter', doc='max number of iterations (>= 0).'): 50},
 {Param(parent='MultilayerPerceptronClassifier_c88cde667a78', name='blockSize', doc='block size for stacking input data in matrices. Data is stacked within partitions. If block size is more than remaining data in a partition then it is adjusted to the size of this data.'): 64,
  Param(parent='MultilayerPerceptronClassifier_c88cde667a78', name='maxIter', doc='max number of iterations (>= 0).'): 100},
 {Param(parent='MultilayerPerceptronClassifier_c88cde667a78', name='blockSize', doc='block size for stacking input data in matrices. Data is stacked within partitions. If block size is more than 

**Run time**

*   50 minutes




In [59]:
mlpModel = mlp_cv.fit(train)

In [61]:
best_model_mlp = mlpModel.bestModel.summary
best_model_mlp

<bound method MultilayerPerceptronClassificationModel.summary of MultilayerPerceptronClassificationModel: uid=MultilayerPerceptronClassifier_c88cde667a78, numLayers=4, numClasses=2, numFeatures=512>

In [60]:
prediction = mlpModel.transform(test)


In [62]:
prediction.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[-3.4943213292353...|  1.0|[-13.098479088909...|[7.40909773523040...|       1.0|
|[-3.2656161487715...|  1.0|[-14.634077254400...|[3.67203031399957...|       1.0|
|[-3.1191907454870...|  1.0|[-14.631298147080...|[3.30309862298269...|       1.0|
|[-2.3243082371492...|  1.0|[-11.504662610200...|[1.92296521756709...|       1.0|
|[-2.3143316976378...|  1.0|[-13.147072432531...|[6.19692544289977...|       1.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
prediction.select('label', 'prediction').show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 10 rows



#**predictions and evaluate model performance**

##**Area Under the Curve**

In [63]:
my_eval_mlp = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_mlp.evaluate(prediction)

0.9799444564334968

##**F1 Score**

In [64]:
f1_score = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
f1_score.evaluate(prediction)

0.9843768881262537

##**Accuracy**

In [65]:
accuracy_mlp = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
accuracy_mlp.evaluate(prediction)

0.9844357976653697

In [66]:
label = prediction.select(['label']).collect()
label_pred = prediction.select(['prediction']).collect()



In [67]:
type(label)

list

In [68]:
print(classification_report(label, label_pred))

              precision    recall  f1-score   support

         0.0       1.00      0.96      0.98       489
         1.0       0.98      1.00      0.99       796

    accuracy                           0.98      1285
   macro avg       0.99      0.98      0.98      1285
weighted avg       0.98      0.98      0.98      1285



In [69]:
print(confusion_matrix(label, label_pred))

[[470  19]
 [  1 795]]


In [70]:
prediction.groupBy('label','prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       0.0|    1|
|  0.0|       0.0|  470|
|  1.0|       1.0|  795|
|  0.0|       1.0|   19|
+-----+----------+-----+



#Model II

*   hyperparameter tunning
*   Train validation split method



In [ ]:
## Parameter Grid
layers = [512,128,64,2]
mlp = MultilayerPerceptronClassifier(layers = layers, seed = 1)
paramGrid_mlp = ParamGridBuilder() \
    .addGrid(mlp.blockSize, {32,64,128}) \
    .addGrid(mlp.maxIter, {50,100,150}) \
    .build()

In [ ]:
tvs = TrainValidationSplit(estimator=mlp,
                           estimatorParamMaps=paramGrid_mlp,
                           evaluator=BinaryClassificationEvaluator(),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)

In [ ]:
model = tvs.fit(train)

#predictions and evaluate model performance

In [ ]:
hold_out_pred = model.transform(test)


In [ ]:
hold_out_pred.select('label', 'prediction').show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 10 rows



#**Accuracy and Test Error**

In [ ]:


evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
accuracy = evaluator.evaluate(hold_out_pred)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

Accuracy = 0.9765114038880166
Test Error = 0.023488596111983395


#**F1 score**

In [ ]:
f1_score = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
f1_score.evaluate(hold_out_pred)

0.9765114038880166

#**confusion matrix**

In [ ]:
true_label = hold_out_pred.select(['label']).collect()
pred_label = hold_out_pred.select(['prediction']).collect()

In [ ]:
cm = print(confusion_matrix(true_label, pred_label))

[[460  29]
 [  1 795]]
